In [24]:
import time
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [44]:
from selenium import webdriver  # 동적크롤링
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys # Enter
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

In [45]:
df = pd.read_csv('근로복지공단_산재보험 지정의료기관 현황정보_20221231.csv', encoding='cp949')
df.head(10)

,연도,의료기관기호,의료기관명,우편번호,주소,전화번호
0,2022,1000001,연세대학교의과대학강남세브란스병원,6273,서울 강남구 언주로 211 (도곡동),02-2019-2163
1,2022,1000050,의료법인 제일병원,6664,서울 서초구 서초대로 116 (방배동),02-586-5557
2,2022,1000072,예루살렘정형외과,6737,서울 서초구 남부순환로350길 33 (양재동),02-575-0122-0
3,2022,1000116,가톨릭정형외과의원,6109,"서울 강남구 언주로 601 3층 (논현동, 파크랜드빌딩)",02-554-7575
4,2022,1000125,안정형외과,6021,서울 강남구 도산대로 303 (신사동),02-3444-7277
5,2022,1000147,광동한방병원,6170,서울 강남구 봉은사로 612 (삼성동),02-2222-4883
6,2022,1000158,한양대학교병원,4763,"서울 성동구 왕십리로 222-1 (사근동, 한양대학부속병원)",02-2290-9096
7,2022,1000159,강동성심병원,5355,서울 강동구 성안로 150 (길동),02-2224-2127
8,2022,1000161,국립경찰병원,5715,서울 송파구 송이로 123 (가락동),02-3400-1160
9,2022,1000168,건국대학교병원,5030,"서울특별시 광진구 능동로 120-1 (화양동, 건국대학교병원)",02-2030-7160


In [64]:
df['의료기관명'][1304]

'중앙대학교 병원'

In [65]:
hospital_idx = int(input())

 1304


In [66]:
# 카카오 맵으로 이동
url = "https://map.kakao.com/"
chrome_driver_path = 'C:/Users/kellykim/Downloads/chromedriver-win64/chromedriver.exe'
service = Service(chrome_driver_path)
options = Options()
options.add_argument('--headless')  # 브라우저를 숨김 모드로 실행
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(service=service, options=options)
driver.get(url)

In [67]:
searchloc = df['의료기관명'][hospital_idx] + ' ' + ' '.join(list(df['주소'][hospital_idx].split())[:2])

search_area = driver.find_element(By.XPATH, r'//*[@id="search.keyword.query"]') # 카카오맵 검색창
search_area.send_keys(searchloc)  # 검색어 전달
driver.find_element(By.XPATH, r'//*[@id="search.keyword.submit"]').send_keys(Keys.ENTER)  # 돋보기 클릭

time.sleep(5)

driver.find_element(By.XPATH, r'//*[@id="info.search.place.list"]/li['+str(1)+']/div[5]/div[4]/a[1]').send_keys(Keys.ENTER)

In [68]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
room = soup.select_one('.placelist > .PlaceItem') # 제일 상단의 장소 정보를 가져옴

In [69]:
registered = 1 # 카카오맵에 의료기관 미등록
review_exist = 1 # 후기 개수 0개 or 후기 미제공

In [70]:
# 의료기관 명
try:
    name = room.select('.head_item > .tit_name > .link_name')[0].text
except AttributeError:
    name = searchloc
    registered = 0 # 카카오맵에 등록되지 않은 의료기관임
    review_exist = 0

In [71]:
name

'중앙대학교병원'

In [72]:
# 의료기관 주소
if registered:
    addr = room.select('.addr > p')[0].text
else:
    addr = df['주소'][hospital_idx]

In [73]:
addr

'서울 동작구 흑석로 102'

In [74]:
# 의료기관 평점
if registered:
    score = room.select('.rating > .score > em')[0].text
    review_cnt = room.select('.rating > a > em')[0].text
    if (score == '' or not int(review_cnt)): #후기 미제공 or 후기 아직 없음
        review_exist = 0
        score = "X"
else:
    score = "X" #카카오맵에 미등록된 의료기관

In [75]:
score

'2.2'

In [76]:
# 리뷰 수집 및 ChatGPT API를 활용한 요약
def extract_review():
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    # 후기 목록 찾기
    review_lists = soup.select('.list_evaluation > li')
    
    count = 0
    rev = []

    if len(review_lists) != 0:
        for review in review_lists:
            comment = review.select('.txt_comment > span')[0].text  # 리뷰
            if len(comment) != 0:
                rev.append(comment)
    
    # 다시 검색 탭으로 전환
    driver.close()
    driver.switch_to.window(driver.window_handles[0])
    time.sleep(2)
    
    return rev
    
if review_exist:
    # '상세보기'로 이동
    driver.find_element(By.XPATH, r'//*[@id="info.search.place.list"]/li['+str(1)+']/div[5]/div[4]/a[1]').send_keys(Keys.ENTER)
    driver.switch_to.window(driver.window_handles[-1])
    time.sleep(2)
    # '리뷰 더보기' 최대 5번 클릭 하기
    for i in range(5):
        try:
            more_rvs = driver.find_element(By.CLASS_NAME, 'txt_more') # 후기 더보기 탭
            more_rvs.click()
            time.sleep(1)
        except NoSuchElementException:
            pass
    
    rev = extract_review()  # 리뷰 추출 함수 실행
else:
    rev = "X"

In [77]:
rev

['살다살다  대학 병원 후기를 적게 될 줄 몰랐는데중앙대병원 전체적으로 간호선생님들과 수납처 의사쌤까지 다 너무 친절하다.진짜 뭔가 배려심도 기본적으로 가지고 계신게 느껴져서기분좋게 원활하게 진료 잘 받았어요.시기가 시기인지라 다른 병원은 예약도 너무 어려웠는데중앙대 병원 관계된 모든 분들께 감사드립니다.(호흡기알레르기 내과, 영상의학과 다녀옴)',
 '시험관시술 의료사고 기사읽고 왔습니다',
 '약물사고로 여태 치유안되는 몸고생이며 타원에서 엄청난관련치료비 지출이며 엄청난 휴유증을 안긴 곳.사소하게는 다른 병원서 안아프게 살짝 피부에 하는 항생제 테스트도 아프게 쑤셔 박듯 한후(간호사왈 다들 아파한다나...잘못해서 그런 것도 모르는듯..) 신경 건드려 오랫동안 손이 저림.아무튼 개인적으로 크고 작은 의료사고가 있던 병원. 실력도 의문ㅡ다소 드문 병변 하나에 대한 지식자체가 아예 없었음(최악은 본인들 무지를 가지고 그런 증상의 병변은 없다느니 환자를 이상한 사람 취급해버림. 나중 의료백과 사전만 뒤져봐도 나오던걸). 당연히 후에  타원에서 진단 받고 이 병원은 다신 안감',
 '혈액종양내과오충렬교수님 항상감사드립니다 저희친정어머님 암때문에불안하고 항상우울해하셨는데웃는얼굴로 인사해주시고 손잡아주시고 따뜻한말씀한마디에 저희어머님 힘든투병생활동안 기운을얻으셨습니다연로하시고 담낭암4기라 결국 돌아가셨지만 입원실에서 자신의마지막을 직감하셨는지 오충렬교수님께감사인사드려야하신다며제게 다짐을시키셨습니다감사한마음을전할곳을 알아보니 김영란법에저촉됀다하여 선물도드리지못하고 바쁜진료시간에 인사드리는것은 다른암환우분의소중란시간을뺏는것같아 이곳에 글을올립니다오충렬교수님과혈액종양내과 귀엽고예쁘신(외모뿐아니라손녀처럼살잘대해주셨습니다)진료간호사님(진료소통을위해 교수님의의사를 전달해주신고마운분인데 성함을 잊었습니다 )과 입원실상주간호사님들 (돌아가시기전6일의입원시간동안 많이애써주셨습니다)머리숙여감사드립니다 앞으로도 길을지나며병원을볼때마다 감사한기억이날것같습니다감사했습니다',
 'HIV 익명

In [78]:
# 참고한 사이트 : https://dduniverse.tistory.com/entry/%EC%B9%B4%EC%B9%B4%EC%98%A4%EB%A7%B5-%EB%A6%AC%EB%B7%B0-%ED%81%AC%EB%A1%A4%EB%A7%81-%ED%95%98%EA%B8%B0-with-Selenium-BeautifulSoup

In [79]:
from openai import OpenAI
 
# OPENAI_YOUR_KEY에 open ai에서 발급받은 api key를 등록합니다.
# 사용 모델을 설정합니다. chat GPT는 gpt-3.5-turbo를 사용합니다.
MODEL = "gpt-3.5-turbo"
USER_INPUT_MSG = "다음의 하나 이상의 병원 리뷰를 요약해서 종합한 하나의 리뷰를 출력해줘. 리뷰 요약을 할 때 주체는 특정인이 아닌 제3자인 것처럼 해줘. 서술어 스타일은 '~입니다'나 '~합니다'와 같이 해줘. 시작은 '" + name + "(병원이름)은'으로 설정해주고 250자 내외로 알려줘. " + str(rev)

# OpenAI API 키 설정
client = OpenAI(api_key=OPENAI_YOUR_KEY)  # 사용자의 API 키로 대체해야 함
if review_exist:
    response = client.chat.completions.create(
        model = MODEL,
        messages = [{"role" : "user", "content" : USER_INPUT_MSG}]
    )
    review_summary = response.choices[0].message.content
else:
    review_summary = "X"

In [80]:
print(review_summary)

중앙대학교병원은 전체적으로 간호선생님들과 수납처 의사들이 매우 친절하다. 예약이 어려운 다른 병원과는 달리 원활하게 진료를 받을 수 있어 좋다고 한다. 그러나 다른 환자들은 의료사고로 인해 치유되지 않거나 피해를 입은 경험을 공유하고 있다. 또한 예약관리가 잘 되지 않는 등 운영에 개선이 필요한 부분도 있다. 전반적으로 의료진은 친절하고 진료는 만족스러운 편이지만, 시설 및 시스템에는 개선이 필요한 부분이 있다고 평가되어진다.
